### Preparing assembly files for binning
To create a depth processing file, reads must be re-aligned to the contigs. This has been done using bowtie2 (can also be done using BWA). The next step would be to create a depth file with MetaBat2, convert that to be suitable for CONCOCT and MaxBin2, and then process these into bins. 

In [1]:
#this creates a depth file for MetaBat
#check names of the mapping files
for f in Coral1 Coral2 Coral4 Coral5 T3-21-Mmea T3-4-Past
do
jgi_summarize_bam_contig_depths --outputDepth ../data/working/metabat_depth_"$f".txt ../../04_mapping/data/results/"$f".bam
done

SyntaxError: invalid syntax (<ipython-input-1-22de35fddd80>, line 3)

In [2]:
#this creates the MaxBin depth file SEE IF THERE IS AN ALTERNATIVE
for f in Coral1 Coral2 Coral4 Coral5 T3-21-Mmea T3-4-Past; do
    jgi_summarize_bam_contig_depths --outputDepth ../data/working/mb2_master_depth_"$f".txt --noIntraDepthVariance ../../04_mapping/data/results/"$f".bam
    A=($(head -n 1 ../data/working/mb2_master_depth_"$f".txt)) 
    N=${#A[*]}
done
#the next bit breaks the depth file into different files for MaxBin to use       
for i in $(seq 4 $N); do 
		sample=$(head -n 1 ../data/working/mb2_master_depth_"$f".txt | cut -f $i) #dont think this line works yet
		grep -v totalAvgDepth ${out}/work_files/mb2_master_depth.txt | cut -f 1,$i > ${out}/work_files/mb2_${sample%.*}.txt
        

SyntaxError: invalid syntax (<ipython-input-2-70357cb867cc>, line 2)

In [6]:
#this creates the CONCOCT depth file

#this part cuts up the contigs into 10kb pieces for CONCOCT to use !Are the chunks too large?! check filenames
for f in Coral1 Coral2 Coral4 Coral5 T3-21-Mmea T3-4-Past
do
cut_up_fasta.py "$f"_assembly.fa -c 1000 -o 0 --merge_last -b ../data/working/"$f"_contigs_10K.bed > ../data/working/"$f"_contigs_cut.fa

#this part estimates contig coverage
concoct_coverage_table.py ../data/working/"$f"_contigs_cut.bed ../../04_mapping/"$f".sorted.bam > coverage_table_"$f".tsv


SyntaxError: invalid syntax (<ipython-input-6-7db03b054bf7>, line 4)

In [5]:
#maxbin2 final script for processing #documentation for maxbin is unavailable, do we want to continue?


In [9]:
#CONCOCT script
#make correct directories (can be omitted I think)
mkdir ../data/results/concoct_bins
mkdir ..data/working/concoct_temp
#run the following for all your samples
for f in Coral1 Coral2 Coral4 Coral5 T3-21-Mmea T3-4-Past
do
#this creates separate directories for all your samples, especially useful at later stages with many samples
mkdir ../data/results/concoct_bins/"$f"_concoct_bins
mkdir ../data/working/concoct_temp/"$f"_concoct_temp
#this next bit actually runs CONCOCT itself
concoct --composition_file ../data/working/"$f"_contigs_cut.fa --coverage_file ../data/working/coverage_table_"$f".tsv -b ../data/working/concoct_temp/"$f"_concoct_temp/
merge_cutup_clustering.py ../data/working/concoct_temp/"$f"_concoct_temp/clustering_gt1000.csv > ..data/working/concoct_temp/"$f"_concoct_temp/"$f"_clustering_merged.csv
mkdir ../data/results/concoct_bins/"$f"_concoct_bins
extract_fasta_bins.py "$f"_assembly.fa ..data/working/concoct_temp/"$f"_concoct_temp/"$f"_clustering_merged.csv --output_path ../data/results/concoct_bins/"$f"_concoct_bins
done

SyntaxError: invalid syntax (<ipython-input-9-1ee438ed8a32>, line 3)